In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np
from PINNs_Chron import PINN1D, PINN1DSolver

import matplotlib.pyplot as plt

# Set data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)
tf.get_logger().setLevel('ERROR')

In [ ]:
%matplotlib inline

SIZE = 12
BIGGER_SIZE = 16

plt.rc('font', family='Arial', size=SIZE) # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize

In [ ]:
def logistic_f(t, u0=0, u1=1, u1a=0, k=1, t1=0):
    return u0 + (u1 - u1a) / (1 + np.exp(-k * (t - t1)))

tf_logistic = lambda t, u0, u1, u1a, k, t1: u0 + (u1 - u1a) * tf.math.sigmoid(k * (t - t1))

In [ ]:
t_end = 100.
h = 30.
Tsurf = 0.
Tbot = 600.
Tgrad0 = (Tbot - Tsurf) / h
kappa = 25.
u0, u1, t1 = .05, .6, 60
u2, t2 = .1, 80
uplift = lambda t : logistic_f(t, u0=u0, u1=u1, u1a=u0, k=1, t1=t1) + logistic_f(t, u0=0, u1=u2, u1a=u1, k=1, t1=t2)
tf_uplift = lambda t: tf_logistic(t, u0, u1, u0, 1, t1) + tf_logistic(t, 0, u2, u1, 1, t2)
tf_uplift_inv = lambda t, u0, u1, t1, u2, t2: tf_logistic(t, u0, u1, u0, 1, t1) + tf_logistic(t, 0, u2, u1, 1, t2)

In [ ]:
# Define the initial condition
def T_init(z):
    return Tbot - Tgrad0 * z

# Define boundary condition
def T_surf(z):
    return tf.constant(Tsurf, shape=z.shape, dtype=DTYPE)

def T_bot(z):
    return tf.constant(Tbot, shape=z.shape, dtype=DTYPE)

In [ ]:
# Set number of data points
N_0 = 100
N_b = 200
N_r = 2000

# Set boundary
tmin = 0.
tmax = t_end
zmin = 0.
zmax = h

# Lower bounds
lb = tf.constant([tmin, zmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([tmax, zmax], dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

# Draw uniform sample points for initial boundary data
t_0 = tf.ones((N_0, 1), dtype=DTYPE) * lb[0]
z_0 = tf.random.uniform((N_0, 1), lb[1], ub[1], dtype=DTYPE)
X_0 = tf.concat([t_0, z_0], axis=1)

# Evaluate intitial condition at z_0
T_0 = T_init(z_0)

# Boundary data
t_b = tf.random.uniform((N_b*2, 1), lb[0], ub[0], dtype=DTYPE)
z_b = lb[1] * tf.ones((N_b, 1), dtype=DTYPE)
z_s = ub[1] * tf.ones((N_b, 1), dtype=DTYPE)
z_bs = tf.concat([z_b, z_s], axis=0)
X_b = tf.concat([t_b, z_bs], axis=1)

# Evaluate boundary condition at (t_b, z_b)
T_b = T_bot(z_b)
T_s = T_surf(z_s)
T_bs = tf.concat([T_b, T_s], axis=0)

# Draw uniformly sampled collocation points
t_r = tf.random.uniform((N_r, 1), lb[0], ub[0], dtype=DTYPE)
z_r = tf.random.uniform((N_r, 1), lb[1], ub[1], dtype=DTYPE)
X_r = tf.concat([t_r, z_r], axis=1)

# Collect boundary and inital data in lists
X_data = tf.concat([X_0, X_b], axis=0)
T_data = tf.concat([T_0, T_bs], axis=0)

In [ ]:
class PINNInv1D(PINN1D):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.u0 = tf.constant(1.)
        self.u1 = tf.constant(1.)
        self.t1 = tf.constant(50.)
        self.u2 = tf.constant(1.)
        self.t2 = tf.constant(50.)

In [ ]:
from scipy.optimize import differential_evolution
from scipy.optimize import shgo
from scipy.optimize import direct
from scipy.optimize import dual_annealing
from time import time
import os

class InvSolver(PINN1DSolver):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.age_loss_hist = []
        self.u0_hist = []
        self.u1_hist = []
        self.t1_hist = []
        self.u2_hist = []
        self.t2_hist = []
        self.inv_it_hist = []
        self.inv_iter = 0
        #the following needs to be set up for optimization
        self.data_age = tf.constant([])
        self.data_method = []
        self.data_radi = []
        self.data_err = []
        self.savepath = 'saved_inv_model'
        
    def fun_u(self, t):
        #this defines the model parameters to optimize
        return tf_uplift_inv(t, self.model.u0, self.model.u1, self.model.t1,
                             self.model.u2, self.model.t2)
            
    def age_loss(self):
        age_pred = []
        sample_t, sample_T = self.get_tT()
        for hm, hr in zip(self.data_method, self.data_radi):
            age_pred.append(self.pred_age(sample_t, sample_T, method=hm, grain_radius=hr))
            
        return tf.reduce_mean(tf.square((age_pred - self.data_age)/self.data_err))

    def solve_with_scipy(self, bounds,
                         samples=100, disp=True, maxiter=100, maxfun=None,
                         optimizer='shgo', sampling_method='simplicial',local_bias=False,
                         strategy='best2bin', mutation=(0.5, 1), recombination=0.7,
                         popsize=30, eps=1e-3):
        def obj_fn(x):
            u0, u1, t1, u2, t2 = x
            if t1 >= t2:
                return 99999.
            else:
                self.model.u0 = u0
                self.model.u1 = u1
                self.model.t1 = t1
                self.model.u2 = u2
                self.model.t2 = t2
                age_loss = self.age_loss()
                self.u0_hist.append(u0)
                self.u1_hist.append(u1)
                self.t1_hist.append(t1)
                self.u2_hist.append(u2)
                self.t2_hist.append(t2)
                self.inv_it_hist.append(self.inv_iter)
                self.current_age_loss = age_loss.numpy()
                self.age_loss_hist.append(self.current_age_loss)
            return age_loss

        def save_inv_history(file):
            np.savez(file, u0_hist=self.u0_hist, u1_hist=self.u1_hist, u2_hist=self.u2_hist,
                     t1_hist=self.t1_hist, t2_hist=self.t2_hist,
                     inv_it_hist=self.inv_it_hist, age_loss_hist=self.age_loss_hist)
               
        def scipy_callback(*xf):
            self.inv_iter += 1
            x = xf[0]
            fun = obj_fn(x)
            runpath = self.get_runpath()
            print('It {:6d} fun {:5.4g}: u0 = {:5.4g} u1 = {:5.4g} t1 = {:6.4g} u2 = {:5.4g} t2 = {:6.4g}'.format(
                                  self.inv_iter , fun, x[0], x[1], x[2], x[3], x[4]))
            save_inv_history(runpath + '/inv_solver_hist')
            self.save_history(runpath + '/solver_hist')
            with open(runpath + '/inv_log.txt', 'a') as log_f:
                log_f.write(f'it {self.inv_iter} - fun: {fun}; x: {x}\n')

            return self.current_loss < 1 and self.current_age_loss < .01
            
        tic = time()
        runpath = self.get_runpath()
        with open(runpath + '/inv_log.txt', 'a') as log_file:
            log_file.write(f'-- search begin -- bounds: {bounds}\n')

        if optimizer=='shgo':
            res = shgo(obj_fn, bounds, n=samples, options={'maxiter':maxiter, 'disp':disp},
                        sampling_method=sampling_method, callback=scipy_callback)
        elif optimizer=='DE':
            res = differential_evolution(obj_fn, bounds, strategy=strategy, init=sampling_method,
                   disp=disp, maxiter=maxiter, popsize=popsize, polish=local_bias,
                   mutation=mutation, recombination=recombination, callback=scipy_callback)
        elif optimizer=='direct':
            res = direct(obj_fn, bounds, maxiter=maxiter, maxfun=maxfun, eps=eps,
                          locally_biased=local_bias, callback=scipy_callback)
        elif optimizer=='DA':
            res = dual_annealing(obj_fn, bounds, maxiter=maxiter, maxfun=maxfun, callback=scipy_callback)
        else:
            raise Exception('optimizer is not available')
            
        with open(runpath + '/inv_log.txt', 'a') as log_file:
            log_file.write(f'-- search ends -- run time: {time() - tic:.0f}; optima: {res.x}\n')
            
        return res

In [ ]:
def read_ages(txt, DYTPE='float32'):
    ar = np.loadtxt(txt, dtype='str')
    methods = ar[:, 0]
    ages = tf.constant(ar[:, 1].astype(DTYPE))
    radi = ar[:, 2].astype(DTYPE)
    return methods, ages, radi

In [ ]:
def l_rate(step, initial_rate=1e-3, decay_rate=.9, decay_steps=1000):
    return initial_rate * decay_rate ** (step / decay_steps)
    
# x = np.linspace(0, 50000, 1000)
# plt.plot(x,  l_rate(x, decay_steps=1000))
# plt.yscale('log')

In [ ]:
# Initialize model
model = PINNInv1D(lb, ub, num_hidden_layers=3,
                   num_neurons_per_layer=20)
model.build(input_shape=(None, 2))

# lr = 1e-3
# lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([2e3, 1e4],[1e-3, 1e-4, 1e-5])
lr = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, decay_steps=1000, decay_rate=0.9)
optim = tf.keras.optimizers.Adam(learning_rate=lr)

# Initilize PINN solver
solver = InvSolver(model, X_r)
solver.savepath = 'saved_inv_model'
solver.data_method, solver.data_age, solver.data_radi = read_ages('1d_ages.txt')
solver.data_err = solver.data_age * .1

In [ ]:
def shrink(b0, x, frac=.9):
    d = (np.max(b0) - np.min(b0)) * frac * .5
    b1l = x - d
    b1u = x + d
    dl = b1l - np.min(b0)
    du = np.max(b0) - b1u
    if dl < 0:
        b1l -= dl
        b1u -= dl
    elif du < 0:
        b1l += du
        b1u += du

    return (b1l, b1u) 
    
def new_bounds(bounds, optima, optima_0, frac=.9):
    new_bounds = []
    for b0, x, op0 in zip(bounds, optima, optima_0):
        r = np.diff(b0)[0]
        if np.abs((x - op0) / r) < .05:
            b1 = shrink(b0, x, frac=frac)
        else:
            b1 = b0
            
        new_bounds.append(b1)
        
    return new_bounds

def zoom_in(bounds, optima, optima_0):
    new_bounds = []
    for b0, x, op0 in zip(bounds, optima, optima_0):
        r = np.diff(b0)[0]
        if np.abs((x - op0) / r) < .05:
            d = np.abs(np.array(b0) - x).min()
            b1 = (x - d, x + d)
        else:
            b1 = b0
            
        new_bounds.append(b1)

    return new_bounds

In [ ]:
def new_adam_n(adam_n, up_by=1.5, max=10000):
    return min(max, np.int32(adam_n * up_by))

In [ ]:
from time import time # Initialize model
# Start timer
tic = time()

bounds = [(0, 2), (0, 2), (0, 100), (0, 2), (0, 100)]
ops = [np.mean(a) for a in bounds]

maxiter, maxfun = 30 + 2, 1000

runpath = solver.get_runpath()

np.save(runpath  + '/inv_X_r', X_r)
adam_n = 100
for i in range(15):
    solver.solve_with_Adam(optim, X_data, T_data, N=adam_n, echofreq=adam_n, savefreq=-1)   
    res = solver.solve_with_scipy(bounds, maxiter=maxiter, maxfun=maxfun,
                                  optimizer='direct', eps=1e-1, local_bias=False
                                  )
    tf.saved_model.save(model, os.path.join(runpath, 'model' + str(i)))
    bounds = new_bounds(bounds, res.x, ops, frac=.8)
    # bounds = zoom_in(bounds, res.x, ops)
    ops = res.x
    adam_n = new_adam_n(adam_n, up_by=1.5, max=10000)
    solver.model.u0, solver.model.u1, solver.model.t1, solver.model.u2, solver.model.t2 = res.x

# Print computation time
print('\nComputation time: {} seconds'.format(time()-tic));